# Aerhan Srirangan's Final Project

# Imports

In [1]:
# necessary imports
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import datetime as dt

from keras.layers import *
from keras.models import Sequential
from keras.utils import plot_model
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


# Specify tensorflow version
print("Tensorflow Version:", tf.__version__)

2023-08-04 08:14:37.985523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow Version: 2.10.0


In [2]:
# Specify current directory
print("Current directory: ", os.getcwd())
# List current directory
print("Directory List", os.listdir(os.getcwd()))
# Specify the directory containing the dataset
DATASET_DIR = "UCF-Crime Dataset"
# Specify path to dataset directory
dataset_path = os.path.join(os.getcwd(), DATASET_DIR)
# Get each category of the dataset
dirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print(dirs)
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64


Current directory:  /Users/aerhansrirangan/Documents/Uni/Year 3/Final Year Project/Model
Directory List ['~$nal Implementation Report.docx', 'UCF-Crime Dataset', 'Final Project.py', '.DS_Store', 'Final Implementation Report.pdf', 'Final_Year_Project.ipynb', 'Final Implementation Report.docx', 'convlstm_model_structure_plot.png', '.ipynb_checkpoints', 'Final Project.ipynb', '.idea']
['Arson', 'Shoplifting', 'Stealing', 'Burglary', 'Road Accidents', 'Fighting', 'Vandalism', 'Explosion', 'Arrest', 'Normal', 'Abuse', 'Robbery', 'Assault', 'Shooting']


# Fetch Dataset

## Video class

In [3]:
class Video:
    def __init__(self, name: str, frames, filepath: str, label: str, frame_count:int):
        self.name = name
        self.frames = frames
        self.filepath = filepath
        self.label = label
        self.frame_count = frame_count
    
    

### Video Length function

In [4]:
# helper function to display the length of the video
def getVideoDuration(frame_count: float, fps: float) -> str:
    """
    Calculates duration of the video in hh:mm:ss format
    :param frame_count: number of frames in the video
    :param fps: frame rate of the video
    :return: a string representing the duration of the video in hh:mm:ss format
    """
    # Calculate the length of the video in seconds
    length = frame_count / fps

    # Calculate the duration in hours, minutes, and seconds
    hours, remainder = divmod(length, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Return the duration as a formatted string
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"


As the dataset is 37GB, I will only use a tiny portion of it (10 videos per label). Otherwise, it will be too extensive to compute.

In [5]:
# extracting frames from video
def extractFrames(video):
    """
    Extracts frames from a video then normalises the frames
    Args:
        video: the video capture from openCV
    Returns:
        The list of normalised frames
    """
    # A list to store the frames of the video
    frames = []
    while video.isOpened():
        ret, frame = video.read()
        # If the frame was not successfully read, break out of the loop
        if not ret:
            break
        # Convert the frame from BGR into RGB format.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # Normalize the resized frame by dividing it with 255 so each pixel value then lies between 0 and 1
        normalised_frame = resized_frame / 255
        frames.append(normalised_frame)
        
    video.release()
    return frames


In [6]:
# function to fetch dataset stored on hard drive as it's 37GB
# and checks if the videos in the dataset are usable or not
def fetchDataset():
    videos = []

    for label in dirs:
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of: {label}')
        label_path = os.path.join(dataset_path, label)
        print("Label path:", label_path)
        files = [f for f in os.listdir(label_path) if f.endswith('.mp4')]
        # Select the first 10 videos of each label
        file_count = 0
        for file in files:
            filepath = os.path.join(label_path, file)
            # Check if the video file can be opened
            video = cv2.VideoCapture(filepath)
            if file_count == 10:
                break
            if not video.isOpened():
                print(f"Could not open video file: {filepath}")
                files.remove(file)
                continue
            else:
                fps = video.get(cv2.CAP_PROP_FPS)
                frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
                duration = getVideoDuration(frame_count, fps)
                print(file, "has ", duration)            
                # Extract the frames of the video
                frames = extractFrames(video)
                video = Video(file, frames, filepath, label, int(frame_count))
                # Add video to videoarray
                videos.append(video)
                
                file_count += 1
    return videos


In [7]:
# trying to make the video length the same, I will split the videos into equal length 
def splitVideo(video, interval):
    """
    Splits a video into separate parts given the interval of frames.
    Args:
        video: The video as an array.
        interval: The interval of frames.
    Returns:
        A list of the split videos.
    """
    split_videos = []
    start = 0
    while start < len(video):
        end = min(start + interval, len(video))
        part = np.stack(video[start:end], axis=0)  # Stack video parts along axis 0
        split_videos.append(part)
        start = end
    return split_videos


In [8]:
videos = fetchDataset()

def getMinFrames(videos):
    frame_counts = [video.frame_count for video in videos]
    return min(frame_counts)

def getMaxFrames(videos):
    frame_counts = [video.frame_count for video in videos]
    return max(frame_counts)

def hcf(x, y):
    while y:
        x, y = y, x % y
    return x

print("Number of videos", len(videos))

min_frames = getMinFrames(videos)
max_frames = getMaxFrames(videos)

print("max number of frames in all the videos", max_frames)
print("min number of frames in all the videos", min_frames)
print(f"HCF of {min_frames} and {max_frames} is {hcf(min_frames, max_frames)}")


        
video_labels = np.array([video.label for video in videos])
print("Shape of video labels:",video_labels.shape)
print("Data type of video labels",video_labels.dtype)
features = np.asarray([video.frames for video in videos])
print("Shape of features:",features.shape)
print("Data type of features",features.dtype)

print(f"features has length of {len(features)}")
print(f"Video Labels has length of {len(video_labels)}")


print("min number of frames in all the videos", getMinFrames(videos))
print("max number of frames in all the videos", getMaxFrames(videos))


Extracting Data of: Arson
Label path: /Users/aerhansrirangan/Documents/Uni/Year 3/Final Year Project/Model/UCF-Crime Dataset/Arson


[ WARN:0@1.452] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@1.452] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Arson014_x264.mp4 has  00:01:28


[ WARN:0@3.557] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@3.557] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Arson015_x264.mp4 has  00:05:41


...

[ WARN:0@444.217] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@444.217] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Shooting041_x264.mp4 has  00:00:46


[ WARN:0@445.361] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ERROR:0@445.361] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/videoio/src/cap.cpp (166) open VIDEOIO(GSTREAMER): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_5a1v4y7k9y/croot/opencv-suite_1676472757237/work/modules/core/src/utils/filesystem.cpp:580: error: (-213:The function/feature is not implemented)  in function 'exists'




Number of videos 140
max number of frames in all the videos 31643
min number of frames in all the videos 308
HCF of 308 and 31643 is 1
Arson014_x264.mp4 has 2640 frames
Arson014_x264.mp4 has 9 parts
Arson015_x264.mp4 has 10232 frames
Arson015_x264.mp4 has 34 parts
Arson020_x264.mp4 has 2428 frames
Arson020_x264.mp4 has 8 parts
Arson021_x264.mp4 has 1442 frames
Arson021_x264.mp4 has 5 parts
Arson003_x264.mp4 has 4014 frames
Arson003_x264.mp4 has 14 parts
Arson002_x264.mp4 has 4441 frames
Arson002_x264.mp4 has 15 parts
Arson037_x264.mp4 has 803 frames
Arson037_x264.mp4 has 3 parts
Arson036_x264.mp4 has 1699 frames
Arson036_x264.mp4 has 6 parts
Arson040_x264.mp4 has 2835 frames
Arson040_x264.mp4 has 10 parts
Arson041_x264.mp4 has 3757 frames
Arson041_x264.mp4 has 13 parts
Shoplifting052_x264.mp4 has 7646 frames
Shoplifting052_x264.mp4 has 25 parts
Shape of video labels: (271,)
Data type of video labels <U14
Shape of features: (271,)
Data type of features object
features has length of 271


/var/folders/zf/qtm614gd67vdlfqky9hy2kc40000gn/T/ipykernel_9580/43381349.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.asarray([video.frames for video in videos])


# Create Test and Train Set

In [9]:
# splitting into train and test sets
features_train, features_test, labels_train, labels_test = train_test_split(features, 
                                                                            video_labels,
                                                                            test_size=0.2, shuffle=True)

features_train = np.array(features_train)
features_test = np.array(features_test)


# Encoding labels

In [10]:
# Convert labels to numerical categories
# Reshape the array to 2D
labels_train = labels_train.reshape(-1, 1)
labels_train = np.array(labels_train)

labels_test = labels_test.reshape(-1, 1)
labels_test = np.array(labels_test)

# Create an instance of the OneHotEncoder
encoder = OneHotEncoder()

# Fit the encoder to the data and transform the data
encoded_labels_train = encoder.fit_transform(labels_train)
encoded_labels_train = np.array(encoded_labels_train.reshape(-1, 1))
# encoded_labels_train = tf.convert_to_tensor(encoded_labels_train.reshape(-1, 1))

encoded_labels_test = encoder.fit_transform(labels_test)
encoded_labels_test = np.array(encoded_labels_train).reshape(-1, 1)
encoded_labels_test = np.array(encoded_labels_test)

I'll be using One Hot Encoding as it is more flexible and since there is no natural order or hierarchy

## Create ConvLSTM model

In [11]:
def create_ConvLSTM_model():
    model = Sequential()

    model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                         return_sequences=True, input_shape=(min_frames, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.05)))

    model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                         return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.05)))

    model.add(ConvLSTM2D(filters=14, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                         return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.05)))

    model.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                         return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))

    model.add(Flatten())

    model.add(Dense(len(dirs), activation="softmax"))

    # Display the models summary.
    model.summary()

    # Return the constructed convlstm model.
    return model


In [12]:
# Construct the required convlstm model.
convlstm_model = create_ConvLSTM_model()
# Display the success message.
print("Model Created Successfully!")
# Plot the structure of the constructed model.
plot_model(convlstm_model, to_file='convlstm_model_structure_plot.png', show_shapes=True, show_layer_names=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)


# Compile the model and specify loss function, optimizer and metrics values to the model
convlstm_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])


2023-08-04 08:22:32.170372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 308, 62, 62, 4)    1024      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 308, 31, 31, 4)   0         
 )                                                               
                                                                 
 time_distributed (TimeDistr  (None, 308, 31, 31, 4)   0         
 ibuted)                                                         
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 308, 29, 29, 8)    3488      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 308, 15, 15, 8)   0         
 3D)                                                             
                                                        

## Training Model

In [13]:
print("Shape of features_train:", np.shape(features_train))
print("Type of features_train:", type(features_train))
print("Type of encoded training labels:" , type(encoded_labels_train))
print("Shape of encoded training labels:", np.shape(encoded_labels_train))

# Start training the model.
convlstm_model_training_history = convlstm_model.fit(features_train, encoded_labels_train, 
                                                      epochs=50,batch_size=4, shuffle=True,
                                                      callbacks=[early_stopping_callback])
# Evaluate the trained model.
model_evaluation_history = convlstm_model.evaluate(features_test, 
                                                   labels_test)

# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
 
# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
 
# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'convlstm_model___Date_Time_' \
                  f'{current_date_time_string}___Loss_' \
                  f'{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.png'
 
# Save your Model.
convlstm_model.save(model_file_name)


Shape of features_train: (216,)
Type of features_train: <class 'numpy.ndarray'>
Type of encoded training labels: <class 'numpy.ndarray'>
()


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

## Plots

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    """
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    """
    
    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]
    
    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))
 
    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label=metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label=metric_name_2)
 
    # Add title to the plot.
    plt.title(str(plot_name))
 
    # Add legend to the plot.
    plt.legend()


In [ ]:
# Visualize the training and validation loss metrics.
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
# Visualize the training and validation accuracy metrics.
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 
            'Total Accuracy vs Total Validation Accuracy')